# Run the below cell to pull the monthly official cash rate from the ABS

In [4]:
# Pull the Official/Overnight Cash Rate (OCR) using readabs
# Note: the cash rate target is published by the RBA; readabs provides an RBA helper.

import importlib.util

if importlib.util.find_spec("readabs") is None:
    %pip -q install readabs

import pandas as pd
import readabs as ra

# Monthly OCR (PeriodIndex). Set monthly=False for daily.
ocr = ra.read_rba_ocr(monthly=True)

overnight_cash_rate = (
    ocr.rename("overnight_cash_rate")
    .to_frame()
    .reset_index()
    .rename(columns={"index": "period"})
)

# Convert Period -> month-end timestamp for easy joins/plotting
overnight_cash_rate["date"] = overnight_cash_rate["period"].dt.to_timestamp(how="end")
overnight_cash_rate = overnight_cash_rate[["date", "overnight_cash_rate"]].sort_values("date")
overnight_cash_rate.to_csv("data/overnight_cash_rate.csv", index=False)

overnight_cash_rate.tail()

,date,overnight_cash_rate
422,2025-10-31 23:59:59.999999,3.60
423,2025-11-30 23:59:59.999999,3.60
424,2025-12-31 23:59:59.999999,3.60
425,2026-01-31 23:59:59.999999,3.60
426,2026-02-28 23:59:59.999999,3.85


# Run the below cell to extract the nominal mortgage rates from the f05hist.csv file from the RBA

In [6]:
# Extract nominal mortgage rate series FILRHLBVS from RBA F05 (Indicator Lending Rates)
# Source file: data/f05hist.csv

import pandas as pd

f05_path = "data/f05hist.csv"
series_id = "FILRHLBVS"  # Housing loans; Banks; Variable; Standard; Owner-occupier

# F05 format:
# - Row 11 (index 10): Series IDs
# - Row 12+ (index 11+): Data, with first column as period like 'Jan-1959'
raw = pd.read_csv(f05_path, header=None)
series_id_row = raw.iloc[10].tolist()

col_names = []
for i, sid in enumerate(series_id_row):
    if i == 0:
        col_names.append("period")
        continue

    sid_str = str(sid).strip() if pd.notna(sid) else ""
    col_names.append(sid_str if sid_str else f"col_{i}")

f05 = raw.iloc[11:].copy()
f05.columns = col_names

if series_id not in f05.columns:
    raise KeyError(f"Series '{series_id}' not found in {f05_path}")

# Parse monthly period to month-end timestamp (aligns with OCR cell)
dt = pd.to_datetime(f05["period"], format="%b-%Y", errors="coerce")
f05["date"] = dt.dt.to_period("M").dt.to_timestamp(how="end")

nominal_mortgage_rate = (
    f05[["date", series_id]]
    .rename(columns={series_id: "nominal_mortgage_rate"})
    .assign(nominal_mortgage_rate=lambda d: pd.to_numeric(d["nominal_mortgage_rate"], errors="coerce"))
    .dropna(subset=["date"])
    .sort_values("date")
    .reset_index(drop=True)
)

nominal_mortgage_rate.to_csv("data/nominal_mortgage_rate.csv", index=False)
nominal_mortgage_rate.tail()

,date,nominal_mortgage_rate
800,2025-09-30 23:59:59.999999,8.02
801,2025-10-31 23:59:59.999999,8.02
802,2025-11-30 23:59:59.999999,8.02
803,2025-12-31 23:59:59.999999,8.02
804,2026-01-31 23:59:59.999999,8.02


# Below pulls the 'Total dwellings excluding refinancing New loan commitments; Value' series

In [ ]:
# Pull ABS series A130268891F (Total dwellings excl. refinancing; new loan commitments; value)
import pandas as pd
import readabs as ra

data, meta = ra.read_abs_series("5601.0", "A130268891F")

# `read_abs_series` returns a DataFrame indexed by a PeriodIndex (here: quarterly Q-DEC).
# Convert that PeriodIndex to an end-of-period timestamp, matching the pattern used for
# `overnight_cash_rate.csv` (end-of-month/end-of-period timestamps).
if isinstance(data.index, pd.PeriodIndex):
    date_index = data.index.to_timestamp(how="end")
else:
    date_index = pd.to_datetime(data.index, errors="coerce")

data_dt = data.copy()
data_dt.index = date_index
data_dt.index.name = "date"

data_dt.rename(columns={"A130268891F": "millions"}, inplace=True)

out = data_dt.reset_index().sort_values("date")
out.to_csv("data/total-new-loan-commitments-value.csv", index=False, date_format="%Y-%m-%d %H:%M:%S.%f")

out.head()

,date,millions
0,2002-09-30 23:59:59.999999,27708.0
1,2002-12-31 23:59:59.999999,28180.8
2,2003-03-31 23:59:59.999999,30750.4
3,2003-06-30 23:59:59.999999,33127.3
4,2003-09-30 23:59:59.999999,37544.2


# Below pulls the new investor commitments value

In [17]:
# Pull ABS series A130268494A (Investor; new loan commitments; value)
import pandas as pd
import readabs as ra

data, meta = ra.read_abs_series("5601.0", "A130268494A")

# Convert PeriodIndex -> end-of-period timestamp (consistent with other exported series)
if isinstance(data.index, pd.PeriodIndex):
    date_index = data.index.to_timestamp(how="end")
else:
    date_index = pd.to_datetime(data.index, errors="coerce")

data_dt = data.copy()
data_dt.index = date_index
data_dt.index.name = "date"

data_dt.rename(columns={"A130268494A": "millions"}, inplace=True)

out = data_dt.reset_index().sort_values("date")
out.to_csv(
    "data/investor-new-loan-commitments-value.csv",
    index=False,
    date_format="%Y-%m-%d %H:%M:%S.%f",
)

out.head()

,date,millions
0,2002-09-30 23:59:59.999999,11009.5
1,2002-12-31 23:59:59.999999,10978.6
2,2003-03-31 23:59:59.999999,12179.5
3,2003-06-30 23:59:59.999999,13117.8
4,2003-09-30 23:59:59.999999,16164.3


# Below pulls the Owner occupier new loan commitments (number, not value)

In [18]:
# Pull ABS series A130268859F (Owner-occupier; new loan commitments; number)
import pandas as pd
import readabs as ra

data, meta = ra.read_abs_series("5601.0", "A130268859F")

# Convert PeriodIndex -> end-of-period timestamp (consistent with other exported series)
if isinstance(data.index, pd.PeriodIndex):
    date_index = data.index.to_timestamp(how="end")
else:
    date_index = pd.to_datetime(data.index, errors="coerce")

data_dt = data.copy()
data_dt.index = date_index
data_dt.index.name = "date"

data_dt.rename(columns={"A130268859F": "number"}, inplace=True)

out = data_dt.reset_index().sort_values("date")
out.to_csv(
    "data/owner-occupier-new-loan-commitments-number.csv",
    index=False,
    date_format="%Y-%m-%d %H:%M:%S.%f",
)

out.head()

,date,number
0,2002-09-30 23:59:59.999999,92365.0
1,2002-12-31 23:59:59.999999,90689.0
2,2003-03-31 23:59:59.999999,88158.0
3,2003-06-30 23:59:59.999999,96998.0
4,2003-09-30 23:59:59.999999,101071.0


# Below pulls the First home buyer new loan commitments number (not value)

In [19]:
# Pull ABS series A130267830J (First home buyer; new loan commitments; number)
import pandas as pd
import readabs as ra

data, meta = ra.read_abs_series("5601.0", "A130267830J")

# Convert PeriodIndex -> end-of-period timestamp (consistent with other exported series)
if isinstance(data.index, pd.PeriodIndex):
    date_index = data.index.to_timestamp(how="end")
else:
    date_index = pd.to_datetime(data.index, errors="coerce")

data_dt = data.copy()
data_dt.index = date_index
data_dt.index.name = "date"

data_dt.rename(columns={"A130267830J": "number"}, inplace=True)

out = data_dt.reset_index().sort_values("date")
out.to_csv(
    "data/first-home-buyer-new-loan-commitments-number.csv",
    index=False,
    date_format="%Y-%m-%d %H:%M:%S.%f",
)

out.head()

,date,number
0,2002-09-30 23:59:59.999999,25782.0
1,2002-12-31 23:59:59.999999,23601.0
2,2003-03-31 23:59:59.999999,21676.0
3,2003-06-30 23:59:59.999999,22213.0
4,2003-09-30 23:59:59.999999,22748.0
